In [1]:
import xarray as xr
import numpy as np
import torch
import matplotlib.pyplot as plt
import sys

path = '/home/akazemi3/Desktop/MB_Lab_Project/'
sys.path.append(path)
from analysis.neural_data_regression.tools.regression import *

from tools.processing import *
from tools.loading import *

import pandas as pd
import seaborn as sns

In [5]:
def get_best_alpha(data_dict,alphas,legend_name_dict=None):
    

    root_path = '/data/atlas/model_scores'
    

    df = pd.DataFrame()
    
    
    index = 0
    for model, regions in data_dict.items():
                
        for a in alphas:

            regression = f'Ridge(alpha={a})' 
                
            for region in regions:

                data = xr.open_dataset(os.path.join(root_path,f'{model}_{regression}'))
                r_values = data.where(data.region == region,drop=True).r_value.values
                mean_r = np.mean(r_values)
                df_tmp =  pd.DataFrame({'mean_score':mean_r,
                                        'alpha':a,
                                        'model':model,
                                        'region':region},index=[index])
                df = pd.concat([df,df_tmp])
                index += 1

    
    df = df.groupby(['region','model','alpha']).mean().reset_index()
    if legend_name_dict is not None:
        df.model = df.model.map(legend_name_dict)
    max_idx = df.groupby(['model','region']).agg({'mean_score':('idxmax')}).reset_index()['mean_score'].tolist()
    df_max_alphas = df.loc[max_idx].reset_index(drop=True)

    return df_max_alphas
    

In [6]:
# dataset = 'naturalscenes_zscored_processed'

# iden = dataset 
# regions = ['V1','V2','V3','V4']
 
# data_dict = {
#              f'alexnet_mp_{iden}_features.2':regions,
#              f'alexnet_mp_{iden}_features.5':regions,
#              f'alexnet_mp_{iden}_features.7':regions,
#              f'alexnet_mp_{iden}_features.9':regions,
#              f'alexnet_mp_{iden}_features.12':regions,
#            }


# legend_name_dict = {
#              f'alexnet_mp_{iden}_features.2':'alexnet_mp_features.2',
#              f'alexnet_mp_{iden}_features.5':'alexnet_mp_features.5',
#              f'alexnet_mp_{iden}_features.7':'alexnet_mp_features.7',
#              f'alexnet_mp_{iden}_features.9':'alexnet_mp_features.9',
#              f'alexnet_mp_{iden}_features.12':'alexnet_mp_features.12',
#            }
                    

# df_best_alpha = get_best_alpha(data_dict,legend_name_dict)

In [6]:
alphas =  [10**a for a in range(5,7)] 

dataset = 'naturalscenes_zscored_processed'

iden = dataset
regions = ['V4'] #['V1','V2','V3','V4']

data_dict = {
             f'model_final_mp_2l_{dataset}':regions,
             f'model_final_mp_{dataset}':regions
            }


legend_name_dict = {
                    f'model_final_mp_{dataset}':'model',
                    f'model_final_mp_2l_{dataset}':'model_2l'
                    }

df_best_alpha = get_best_alpha(data_dict,alphas,legend_name_dict)


In [ ]:
root_path = '/data/atlas/model_scores'
model = f'model_final_mp_RGB_{iden}'
regression = 'Regression(alpha=0)'
df = xr.open_dataset(os.path.join(root_path,f'{model}{regression}'))
df

<xarray.Dataset>
Dimensions:  (r_values: 3982)
Coordinates:
    subject  (r_values) float64 ...
    region   (r_values) object ...
Dimensions without coordinates: r_values
Data variables:
    r_value  (r_values) float64 ...
    name     object ...

In [31]:
dict_max_alpha = df_best_alpha.set_index(['region','model']).to_dict()['alpha']


In [32]:
dict_max_alpha

{('IT', 'alexnet_mp'): 10000,
 ('V4', 'alexnet_mp'): 10000,
 ('IT', 'alexnet_untrained_mp'): 0,
 ('V4', 'alexnet_untrained_mp'): 1}